In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import pivot_table
import json
from statistics import median

In [ ]:
### Build dataset for Student By Country visualisation

In [2]:
oeu = pd.read_csv('he_map_eu.csv', skiprows=14, thousands=',')
ooe = pd.read_csv('he_map_oe.csv', skiprows=14, thousands=',')
oaf = pd.read_csv('he_map_af.csv', skiprows=14, thousands=',')
oas = pd.read_csv('he_map_as.csv', skiprows=14, thousands=',')
oau = pd.read_csv('he_map_au.csv', skiprows=14, thousands=',')
ome = pd.read_csv('he_map_me.csv', skiprows=14, thousands=',')
ona = pd.read_csv('he_map_na.csv', skiprows=14, thousands=',')
osa = pd.read_csv('he_map_sa.csv', skiprows=14, thousands=',')

In [3]:
aeu = oeu.copy()
aoe = ooe.copy()
aaf = oaf.copy()
aas = oas.copy()
aau = oau.copy()
ame = ome.copy()
ana = ona.copy()
asa = osa.copy()

aeu = aeu.apply(pd.to_numeric, errors='ignore')
aoe = aoe.apply(pd.to_numeric, errors='ignore')
aaf = aaf.apply(pd.to_numeric, errors='ignore')
aas = aas.apply(pd.to_numeric, errors='ignore')
aau = aau.apply(pd.to_numeric, errors='ignore')
ame = ame.apply(pd.to_numeric, errors='ignore')
ana = ana.apply(pd.to_numeric, errors='ignore')
asa = asa.apply(pd.to_numeric, errors='ignore')

cont_list=[aeu,aoe,aaf,aas,aau,ame,ana,asa]

for i in cont_list:
    print(i.shape)

(166, 41)
(166, 30)
(166, 61)
(166, 37)
(166, 32)
(166, 19)
(166, 45)
(166, 18)


In [4]:
# all Uni names are unique
len(aas['HE Provider'].unique())

166

In [5]:
# varify skip rows are consistent
for i in cont_list:
    print(i.columns[0])

UKPRN
UKPRN
UKPRN
UKPRN
UKPRN
UKPRN
UKPRN
UKPRN


In [6]:
# drop row Total
for i in cont_list:
    i.drop([i.index[-1]], inplace=True)
    print(i.shape)

(165, 41)
(165, 30)
(165, 61)
(165, 37)
(165, 32)
(165, 19)
(165, 45)
(165, 18)


In [7]:
# drop first column and last 3 columns
for i in cont_list:
    i.drop([i.columns[0], i.columns[-3], i.columns[-2], i.columns[-1]], axis=1, inplace=True)
    print(i.shape)

(165, 37)
(165, 26)
(165, 57)
(165, 33)
(165, 28)
(165, 15)
(165, 41)
(165, 14)


In [8]:
aas.head(2)

,HE Provider,Afghanistan,Bangladesh,Bhutan,Brunei,Burma [Myanmar],Cambodia,China,East Timor [Timor Leste],Hong Kong (Special Administrative Region of China) [Hong Kong],...,Pakistan,Philippines,Singapore,Sri Lanka,"Taiwan [Taiwan, Province of China]",Tajikistan,Thailand,Turkmenistan,Uzbekistan,Vietnam [Viet Nam]
0,The University of Aberdeen,0,10,0,5,5,0,325,0,20,...,25,5,40,10,5,0,70,0,0,10
1,University of Abertay Dundee,0,0,0,0,0,0,10,0,0,...,5,0,5,0,0,0,0,0,0,0


In [9]:
col_count = 1

for i in cont_list:
    
    col_count += (i.shape[1]-1)
    
print(col_count)

244


In [10]:
amerge = aeu.copy()

for i in cont_list[1:]:
    amerge = pd.merge(amerge, i, on='HE Provider')
           
print(amerge.shape)
amerge.head(2)

(165, 244)


,HE Provider,Austria,Belgium,Bulgaria,Canary Islands,Croatia,Cyprus (European Union),Cyprus not otherwise specified,Czech Republic,Denmark,...,Chile,Colombia,Ecuador,Falkland Islands [Falkland Islands (Malvinas)],Guyana,Paraguay,Peru,Suriname,Uruguay,"Venezuela [Venezuela, Bolivarian Republic of]"
0,The University of Aberdeen,45,55,180,5,10,50,0,100,50,...,0,10,5,0,5,0,5,0,0,0
1,University of Abertay Dundee,5,5,30,0,0,10,0,15,5,...,0,0,0,0,0,0,0,0,0,0


## Cross Check with JSON

In [11]:
with open("world_layers.json") as geofile:
    geojson_layer = json.load(geofile)

In [12]:
geo_list = []

for i in geojson_layer['features']:
    geo_list.append(i.get('properties').get('name'))
    
print(len(geo_list))
print(geo_list[:5])

175
['Bahamas', 'Cuba', 'Canada', 'Dominican Rep.', 'Greenland']


In [13]:
he_list = list(amerge.columns[1:])
print(he_list[:5])

['Austria', 'Belgium', 'Bulgaria', 'Canary Islands', 'Croatia']


In [14]:
# Names in json not in HE
print(len([item for item in geo_list if item not in he_list]))
[item for item in geo_list if item not in he_list][:20]

32


['Bahamas',
 'Dominican Rep.',
 'Bosnia and Herz.',
 'Czech Rep.',
 'United Kingdom',
 'Moldova',
 'Macedonia',
 'Russia',
 'Solomon Is.',
 'Falkland Is.',
 'Venezuela',
 'Iran',
 'Cyprus',
 'N. Cyprus',
 'Korea',
 'Lao PDR',
 'Myanmar',
 'Palestine',
 'Dem. Rep. Korea',
 'Syria']

In [15]:
# Names in HE not in json
print(len([item for item in he_list if item not in geo_list]))
[item for item in he_list if item not in geo_list][:20]

100


['Canary Islands',
 'Cyprus (European Union)',
 'Cyprus not otherwise specified',
 'Czech Republic',
 'French Guiana',
 'Gibraltar',
 'Guadeloupe',
 'Malta',
 'Martinique',
 'Mayotte',
 'Réunion',
 'Åland Islands',
 'Andorra',
 'Bosnia and Herzegovina',
 'Cyprus (Non-European Union)',
 'Faroe Islands',
 'Liechtenstein',
 'Macedonia [Macedonia, The Former Yugoslav Republic of]',
 'Moldova [Moldova, Republic of]',
 'Monaco']

## Clean HE Names

In [16]:
anew = amerge.copy()

In [17]:
col_zero = anew.sum(axis = 0)
col_zero = col_zero[col_zero == 0]

print(len(col_zero))

74


In [18]:
col_zero[:10]

French Guiana                                   0
Guadeloupe                                      0
Martinique                                      0
Mayotte                                         0
Åland Islands                                   0
Andorra                                         0
Liechtenstein                                   0
San Marino                                      0
Svalbard and Jan Mayen                          0
Vatican City [Holy See (Vatican City State)]    0
dtype: object

In [19]:
# drop all country columns with zeros
anew.drop(list(col_zero.index), axis=1, inplace=True)
print(anew.shape)

(165, 170)


In [20]:
anew.columns[anew.columns.str.contains('otherwise')]

Index(['Cyprus not otherwise specified'], dtype='object')

In [21]:
# will want to merge columns with Cyprus
cyprus_col = list(anew.columns[anew.columns.str.contains('Cyprus')])
anew.columns[anew.columns.str.contains('Cyprus')]

Index(['Cyprus (European Union)', 'Cyprus not otherwise specified',
       'Cyprus (Non-European Union)'],
      dtype='object')

In [22]:
anew[cyprus_col].sum()

Cyprus (European Union)           8690
Cyprus not otherwise specified     185
Cyprus (Non-European Union)         55
dtype: int64

In [23]:
anew['Cyprus'] = anew['Cyprus (European Union)'] + \
                    anew['Cyprus not otherwise specified'] + \
                    anew['Cyprus (Non-European Union)']

anew.drop(cyprus_col, axis=1, inplace=True)

print(anew.shape)
print(anew['Cyprus'].sum())

(165, 168)
8930


In [24]:
for i in list(anew.columns):
    if i.replace(" ", "").isalpha() == False:
        print(i)

Macedonia [Macedonia, The Former Yugoslav Republic of]
Moldova [Moldova, Republic of]
Russia [Russian Federation]
Congo (Democratic Republic) [Congo (The Democratic Republic of the)]
Gambia, The
Ivory Coast [Côte D'ivoire]
St Helena, Ascension and Tristan da Cunha
Tanzania [Tanzania, United Republic of]
Burma [Myanmar]
Hong Kong (Special Administrative Region of China) [Hong Kong]
Korea (South) [Korea, Republic of]
Laos [Lao People's Democratic Republic]
Macao (Special Administrative Region of China) [Macao]
Taiwan [Taiwan, Province of China]
Vietnam [Viet Nam]
Iran [Iran, Islamic Republic of]
Occupied Palestinian Territories [Palestine, State of]
Syria [Syrian Arab Republic]
Bahamas, The
British Virgin Islands [Virgin Islands, British]
Falkland Islands [Falkland Islands (Malvinas)]
Venezuela [Venezuela, Bolivarian Republic of]


In [25]:
anew.rename(columns={
    'Macedonia [Macedonia, The Former Yugoslav Republic of]':'Macedonia',
    'Moldova [Moldova, Republic of]': 'Moldova',
    'Russia [Russian Federation]': 'Russia',
    'Congo (Democratic Republic) [Congo (The Democratic Republic of the)]': 'Dem. Rep. Congo',
    'Gambia, The': 'Gambia',
    "Ivory Coast [Côte D'ivoire]": "Côte d'Ivoire",
    'Tanzania [Tanzania, United Republic of]': 'Tanzania',
    'Burma [Myanmar]': 'Myanmar',
    'Korea (South) [Korea, Republic of]': 'Korea',
    "Laos [Lao People's Democratic Republic]": "Lao PDR",
    'Taiwan [Taiwan, Province of China]': 'Taiwan',
    'Vietnam [Viet Nam]': 'Vietnam',
    'Iran [Iran, Islamic Republic of]': 'Iran',
    'Occupied Palestinian Territories [Palestine, State of]': 'Palestine',
    'Syria [Syrian Arab Republic]': 'Syria',
    'Bahamas, The': 'Bahamas',
    'Falkland Islands [Falkland Islands (Malvinas)]': 'Falkland Is.',
    'Venezuela [Venezuela, Bolivarian Republic of]': 'Venezuela'
    }, inplace=True)

anew.drop(["St Helena, Ascension and Tristan da Cunha"], axis=1, inplace=True)
anew.drop(["British Virgin Islands [Virgin Islands, British]"], axis=1, inplace=True)

print(anew.shape)

(165, 166)


In [26]:
for i in list(anew.columns):
    if i.replace(" ", "").isalpha() == False:
        print(i)

Dem. Rep. Congo
Côte d'Ivoire
Hong Kong (Special Administrative Region of China) [Hong Kong]
Macao (Special Administrative Region of China) [Macao]
Falkland Is.


In [27]:
print(anew['China'].sum())
print(anew['Hong Kong (Special Administrative Region of China) [Hong Kong]'].sum())
print(anew['Macao (Special Administrative Region of China) [Macao]'].sum())

101915
15755
650


In [28]:
anew['China'] = anew['China'] + \
                    anew['Hong Kong (Special Administrative Region of China) [Hong Kong]'] + \
                    anew['Macao (Special Administrative Region of China) [Macao]']

anew.drop(['Hong Kong (Special Administrative Region of China) [Hong Kong]',
           'Macao (Special Administrative Region of China) [Macao]'], axis=1, inplace=True)

print(anew.shape)
print(anew['China'].sum())

(165, 164)
118320


In [29]:
# Names in json 
# Not in HE
# These are OK with gray out
print(len([item for item in geo_list if item not in list(anew.columns)]))
[item for item in geo_list if item not in list(anew.columns)]

36


['Cuba',
 'Dominican Rep.',
 'Greenland',
 'Honduras',
 'Haiti',
 'Bosnia and Herz.',
 'Czech Rep.',
 'United Kingdom',
 'Fiji',
 'New Caledonia',
 'Solomon Is.',
 'Papua New Guinea',
 'Vanuatu',
 'Suriname',
 'Bhutan',
 'N. Cyprus',
 'Dem. Rep. Korea',
 'Tajikistan',
 'Timor-Leste',
 'Burundi',
 'Burkina Faso',
 'Central African Rep.',
 "CÃ´te d'Ivoire",
 'Congo',
 'Gabon',
 'Guinea-Bissau',
 'Guinea',
 'Eq. Guinea',
 'Lesotho',
 'Mali',
 'Niger',
 'S. Sudan',
 'W. Sahara',
 'Somaliland',
 'Chad',
 'Togo']

In [30]:
# potential mismatch due to acronyms
for i in [item for item in geo_list if item not in list(anew.columns)]:
    if i.replace(" ", "").isalpha() == False:
        print(i)

Dominican Rep.
Bosnia and Herz.
Czech Rep.
Solomon Is.
N. Cyprus
Dem. Rep. Korea
Timor-Leste
Central African Rep.
CÃ´te d'Ivoire
Guinea-Bissau
Eq. Guinea
S. Sudan
W. Sahara


In [31]:
# Names in HE
# Not in json
print(len([item for item in list(anew.columns) if item not in geo_list]))
[item for item in list(anew.columns) if item not in geo_list]

25


['HE Provider',
 'Canary Islands',
 'Czech Republic',
 'Gibraltar',
 'Malta',
 'Réunion',
 'Bosnia and Herzegovina',
 'Faroe Islands',
 'Monaco',
 "Côte d'Ivoire",
 'Mauritius',
 'Seychelles',
 'South Sudan',
 'Maldives',
 'Singapore',
 'Christmas Island',
 'Bahrain',
 'Anguilla',
 'Antigua and Barbuda',
 'Barbados',
 'Bermuda',
 'Cayman Islands',
 'Dominican Republic',
 'St Lucia',
 'Turks and Caicos Islands']

In [32]:
anew.rename(columns={
    'Czech Republic':'Czech Rep.',
    'Bosnia and Herzegovina':'Bosnia and Herz.',
    'South Sudan':'S. Sudan',
    'Dominican Republic':'Dominican Rep.'
    }, inplace=True)

anew.drop(['Gibraltar'], axis=1, inplace=True)
anew.drop(['Réunion'], axis=1, inplace=True)
anew.drop(['Faroe Islands'], axis=1, inplace=True)
anew.drop(['Christmas Island'], axis=1, inplace=True)
anew.drop(['Anguilla'], axis=1, inplace=True)
anew.drop(['Bermuda'], axis=1, inplace=True)
anew.drop(['Cayman Islands'], axis=1, inplace=True)
anew.drop(['Turks and Caicos Islands'], axis=1, inplace=True)

anew['Spain'] = anew['Spain'] + anew['Canary Islands']

anew.drop(['Canary Islands'], axis=1, inplace=True)

In [33]:
print(len([item for item in list(anew.columns) if item not in geo_list]))
[item for item in list(anew.columns) if item not in geo_list]

12


['HE Provider',
 'Malta',
 'Monaco',
 "Côte d'Ivoire",
 'Mauritius',
 'Seychelles',
 'Maldives',
 'Singapore',
 'Bahrain',
 'Antigua and Barbuda',
 'Barbados',
 'St Lucia']

In [34]:
print(anew.shape)
anew.head(2)

(165, 155)


,HE Provider,Austria,Belgium,Bulgaria,Croatia,Czech Rep.,Denmark,Estonia,Finland,France,...,Chile,Colombia,Ecuador,Falkland Is.,Guyana,Paraguay,Peru,Uruguay,Venezuela,Cyprus
0,The University of Aberdeen,45,55,180,10,100,50,40,295,175,...,0,10,5,0,5,0,5,0,0,50
1,University of Abertay Dundee,5,5,30,0,15,5,5,25,115,...,0,0,0,0,0,0,0,0,0,10


In [35]:
aoutput = anew.melt(id_vars=['HE Provider'],
                  var_name='Country',
                  value_name='Num')

aoutput.rename(columns={'HE Provider':'School'}, inplace=True)

aoutput['Rank'] = aoutput.groupby('Country')['Num'].rank(ascending=True, method='dense')

print(aoutput.shape)
aoutput.head()

(25410, 4)


,School,Country,Num,Rank
0,The University of Aberdeen,Austria,45,8.0
1,University of Abertay Dundee,Austria,5,2.0
2,Aberystwyth University,Austria,0,1.0
3,AECC University College,Austria,0,1.0
4,Anglia Ruskin University,Austria,10,3.0


In [36]:
# test sat
arank = aoutput[aoutput.Country == 'Austria']
arank = arank.sort_values(by=['Rank'], ascending=False)

arank.head(10)

,School,Country,Num,Rank
151,University College London,Austria,105,15.0
49,The University of Edinburgh,Austria,105,15.0
71,King's College London,Austria,90,14.0
90,London School of Economics and Political Science,Austria,75,13.0
154,The University of Warwick,Austria,70,12.0
105,The University of Oxford,Austria,65,11.0
26,The University of Cambridge,Austria,65,11.0
68,"Imperial College of Science, Technology and Me...",Austria,60,10.0
110,Queen Mary University of London,Austria,50,9.0
0,The University of Aberdeen,Austria,45,8.0


In [37]:
arank = aoutput[aoutput.Country == 'Nigeria']
arank[arank.Rank == arank.Rank.max()]

,School,Country,Num,Rank
11092,Coventry University,Nigeria,505,43.0


In [38]:
arank = aoutput[aoutput.Country == 'China']
arank[arank.Rank >= 100]

,School,Country,Num,Rank
14438,The University of Liverpool,China,4940,100.0
14449,The University of Manchester,China,5280,101.0
14506,University College London,China,5640,102.0


In [39]:
aoutput.shape

(25410, 4)

In [40]:
alean = aoutput.copy()

alean = alean[alean['Num'] != 0]

print(alean.shape)
alean.head()

(8999, 4)


,School,Country,Num,Rank
0,The University of Aberdeen,Austria,45,8.0
1,University of Abertay Dundee,Austria,5,2.0
4,Anglia Ruskin University,Austria,10,3.0
5,Aston University,Austria,10,3.0
6,Bangor University,Austria,5,2.0


In [41]:
enrol = pd.read_csv('he_enrolment.csv', skiprows=14, thousands=',')

print(enrol.shape)
enrol.head(2)

(261, 13)


,UKPRN,Provider,England,Wales,Scotland,Northern Ireland,Other UK,Total UK,Other European Union,Non-European Union,Total Non-UK,Not known,Total
0,10008071.0,AA School of Architecture,90,0,0,0,0,90,125,470,595,0,685
1,10007783.0,The University of Aberdeen,1245,30,6310,165,10,7755,2820,1845,4670,0,12425


In [42]:
enrol = enrol[['Provider', 'Total']]

enrol.rename(columns={'Provider': 'School', 'Total': 'Student Total'}, inplace=True)

enrol.head(2)

,School,Student Total
0,AA School of Architecture,685
1,The University of Aberdeen,12425


In [43]:
print(alean.shape)

alean = pd.merge(alean, enrol, on='School')
print(alean.shape)
alean.head(2)

(8999, 4)
(8999, 5)


,School,Country,Num,Rank,Student Total
0,The University of Aberdeen,Austria,45,8.0,12425
1,The University of Aberdeen,Belgium,55,12.0,12425


In [44]:
alean['Perc'] = alean['Num'] / alean['Student Total']
alean['Perc'] = round(alean['Perc'], 4)

alean.head()

,School,Country,Num,Rank,Student Total,Perc
0,The University of Aberdeen,Austria,45,8.0,12425,0.0036
1,The University of Aberdeen,Belgium,55,12.0,12425,0.0044
2,The University of Aberdeen,Bulgaria,180,28.0,12425,0.0145
3,The University of Aberdeen,Croatia,10,3.0,12425,0.0008
4,The University of Aberdeen,Czech Rep.,100,15.0,12425,0.0080


In [45]:
alean[alean.Perc == alean.Perc.max()]

,School,Country,Num,Rank,Student Total,Perc
3783,The University of Liverpool,China,4940,100.0,25370,0.1947


In [46]:
def perc_text(x):
    return "{}%".format(round(x*100, 2))

In [47]:
alean['PercText'] = alean['Perc'].apply(perc_text)
alean['PercRank'] = alean.groupby('Country')['Perc'].rank(ascending=True, method='dense')

print(alean.shape)
alean.head()

(8999, 8)


,School,Country,Num,Rank,Student Total,Perc,PercText,PercRank
0,The University of Aberdeen,Austria,45,8.0,12425,0.0036,0.36%,28.0
1,The University of Aberdeen,Belgium,55,12.0,12425,0.0044,0.44%,33.0
2,The University of Aberdeen,Bulgaria,180,28.0,12425,0.0145,1.45%,62.0
3,The University of Aberdeen,Croatia,10,3.0,12425,0.0008,0.08%,7.0
4,The University of Aberdeen,Czech Rep.,100,15.0,12425,0.0080,0.8%,35.0


In [48]:
# test sat
arank = alean[alean.Country == 'Austria']
arank = arank.sort_values(by=['PercRank'], ascending=False)

arank.head()

,School,Country,Num,Rank,Student Total,Perc,PercText,PercRank
2418,The National Film and Television School,Austria,5,2.0,305,0.0164,1.64%,36.0
4181,London School of Hygiene and Tropical Medicine,Austria,5,2.0,700,0.0071,0.71%,35.0
4081,London School of Economics and Political Science,Austria,75,13.0,11185,0.0067,0.67%,34.0
5520,Royal College of Music,Austria,5,2.0,805,0.0062,0.62%,33.0
2785,Guildhall School of Music and Drama,Austria,5,2.0,980,0.0051,0.51%,32.0


In [49]:
# test sat
arank = alean[alean.Country == 'China']
arank = arank.sort_values(by=['PercRank'], ascending=False)

arank.head()

,School,Country,Num,Rank,Student Total,Perc,PercText,PercRank
3783,The University of Liverpool,China,4940,100.0,25370,0.1947,19.47%,138.0
3001,"Imperial College of Science, Technology and Me...",China,2990,93.0,16925,0.1767,17.67%,137.0
5494,Royal College of Art,China,360,38.0,2040,0.1765,17.65%,136.0
3861,"University of the Arts, London",China,3170,95.0,18190,0.1743,17.43%,135.0
6539,University College London,China,5640,102.0,32710,0.1724,17.24%,134.0


In [50]:
# alean.to_csv('he_map_lean_country.csv')

## Ranking Countries

In [58]:
a_piv = pivot_table(alean,  values='Num', index=['Country'], aggfunc=np.sum)

a_piv['Rank'] = a_piv['Num'].rank(ascending=False, method='dense')
a_piv.sort_values(by=['Rank'], ascending=True, inplace=True)

a_piv.head()

,Num,Rank
Country,,
China,118320,1.0
India,18275,2.0
United States,15945,3.0
Malaysia,14555,4.0
Italy,12735,5.0


In [61]:
int(a_piv.loc['China'].Rank.item())

1

## Calibrating zoom

In [63]:
def flatten(items, seqtypes=(list, tuple)):
    new_items = items.copy()
    int_items = []
    for i, x in enumerate(new_items):
        while i < len(new_items) and isinstance(new_items[i], seqtypes):
            new_items[i:i+1] = new_items[i]
    for i in new_items:
        if isinstance(i, int):
            int_items.append(i)
    return new_items

In [65]:
for i in geojson_layer['features']:
    print("{} - {}".format(i.get('properties').get('name'), len(flatten(i.get('geometry').get('coordinates')))))

Bahamas - 42
Cuba - 84
Canada - 1584
Dominican Rep. - 52
Greenland - 264
Belize - 40
Guatemala - 70
Honduras - 114
Haiti - 42
Costa Rica - 72
Jamaica - 22
Mexico - 340
Puerto Rico - 18
Nicaragua - 104
Panama - 104
El Salvador - 40
United States - 888
Trinidad and Tobago - 16
Albania - 44
Austria - 74
Bosnia and Herz. - 44
Bulgaria - 56
Belgium - 30
Belarus - 88
Czech Rep. - 70
Germany - 116
Switzerland - 48
Spain - 102
Denmark - 48
Finland - 76
Estonia - 34
France - 148
United Kingdom - 112
Croatia - 84
Iceland - 40
Ireland - 26
Greece - 108
Italy - 174
Kosovo - 42
Lithuania - 38
Hungary - 62
Luxembourg - 14
Moldova - 54
Latvia - 44
Macedonia - 34
Netherlands - 28
Norway - 176
Portugal - 66
Montenegro - 36
Russia - 1196
Poland - 90
Slovakia - 66
Sweden - 80
Serbia - 92
Romania - 88
Slovenia - 36
Ukraine - 196
Australia - 482
Fiji - 44
New Caledonia - 26
Solomon Is. - 74
Papua New Guinea - 162
New Zealand - 132
Vanuatu - 24
Brazil - 406
Ecuador - 66
Bolivia - 120
Chile - 228
Peru - 152


## Calc best lat lon

In [92]:
# select US
cn = [i for i in geojson_layer['features'] if (i.get('properties').get('name') == 'United States')][0]

In [93]:
# Viz US flattened lon lat
flatten(cn.get('geometry').get('coordinates'))[:7]

[-155.54211, 19.08348, -155.68817, 18.91619, -155.93665, 19.05939, -155.90806]

In [94]:
# calculate US media lon
median(flatten(cn.get('geometry').get('coordinates'))[0::2])

-123.88204999999999

In [98]:
# calculate US median lat
median(flatten(cn.get('geometry').get('coordinates'))[1::2])

45.07286500000005